In [1]:
  # Fix the strike price 
# Main thing 
# Create samples such that ;
# '''
# The four variables make a sample x is a data point x = (t, S, r, sigma) is data points. 
# No stable results -> then normalise the varaibles. 
#        t -> (0,2) only discrete number of days / 365. T is measure in years. 
#        S -> (50,150) stock prices. 
#        r -> [0%,1%] 
#        sigma (volatility) -> [1%,50%] -> standard deviation of the log of the returns. 
# We will use uniform distribution. 
# '''



In [2]:
pip install pytorch-lightning # https://pytorch-lightning.readthedocs.io/en/latest/common/optimizers.html - optimization to manually turn off pytorch. 


In [3]:
import numpy as np
import scipy.stats as si
import sympy as sy
from sympy.stats import Normal, cdf
from sympy import init_printing
init_printing()
import torch 
from torch import autograd
from torch import nn # import neural network 
from torch import optim 
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import pytorch_lightning as pl

is_cuda = torch.cuda.is_available()

if False:
  device = torch.device("cuda")
  print("GPU is available to use")

else:
  device = torch.device("cpu")
  print( " GPU not availiable to use, CPU used")


 GPU not availiable to use, CPU used


In [4]:
# Increase data up 

In [5]:
# stock price - between [50,150]

# stock price - between [50,150]
torch.set_printoptions(precision =8) # https://pytorch.org/docs/stable/generated/torch.set_printoptions.html - gives us values for options
a,b = 500 ,1 #dimension of the pytorch tensor to be generated
low , high = 50 , 150 #range of uniform distribution
torch.manual_seed(50) #Fx the values
S_t = torch.distributions.uniform.Uniform(low, high).sample([a,b]) # samples from low and high values. a,b are the dimensions

Stock = S_t.view(500,1)
print(Stock.shape)

torch.Size([500, 1])


In [6]:

# Time -[ between 0 and 2]
torch.set_printoptions(precision =8) # fixes the precision
a,b = 500 ,1 #dimension of the pytorch tensor to be generated
low_t , high_t = 0, 730 #range of uniform distribution by number of days [0, 2]Years
torch.manual_seed(56) #Fx the values
t_t= torch.randint(low_t, high_t, size= (500,1) )  # https://pytorch.org/docs/stable/generated/torch.randint.html - generates uniformaly integers

t = t_t / 365 # to get a more reliable spread of results

# time for maturity 
#print(t)
Time = t.view(500,1)
print(Time.shape)

torch.Size([500, 1])


In [7]:
torch.set_printoptions(precision =8)
a,b = 500 ,1 #dimension of the pytorch tensor to be generated
low , high = 0 , 0.01 #range of uniform distribution [0%,1%]
torch.manual_seed(12) #Fx the values
r = torch.distributions.uniform.Uniform(low, high).sample([a,b]) # samples from low and high values. a,b are the dimensions

Rate = r.view(500,1)
print(Rate.shape)

torch.Size([500, 1])


In [8]:
# volatility 
torch.set_printoptions(precision =8)
a,b = 500 ,1 #dimension of the pytorch tensor to be generated
low , high =  0.01, 0.5 #range of uniform distribution [1%,50%]
torch.manual_seed(30) #Fx the values
si = torch.distributions.uniform.Uniform(low, high).sample([a,b]) # samples from low and high values. a,b are the dimensions
# whenevr we want to fix a distribution we just use torch.manual_seed to fix the distribution.
Sigma = si.view(500,1)
print(Sigma.shape) # 500 samples by 1 column

torch.Size([500, 1])


In [9]:
# Stock, Time, Rate ,Sigma -> 4 features (variables)
# Strike
# https://discuss.pytorch.org/t/tensor-stack-or-concatenate/34331/6 - column and concatinate 

X = torch.column_stack([Stock, Time, Rate, Sigma]) # rate = 0 change
print(X)
print(X.shape) # m x n matrix 


tensor([[1.11802200e+02, 7.09589064e-01, 4.65687457e-03, 4.51364458e-01],
        [5.68693161e+01, 1.86849320e+00, 2.32766801e-03, 3.75717849e-01],
        [8.89275208e+01, 1.33424652e+00, 4.52720979e-03, 2.41090730e-01],
        ...,
        [5.91279984e+01, 1.78356159e+00, 2.72788946e-03, 4.56600308e-01],
        [9.81553192e+01, 7.06849337e-01, 1.80347974e-03, 2.50396580e-01],
        [1.06527763e+02, 1.70684934e+00, 2.57011596e-03, 3.33363444e-01]])
torch.Size([500, 4])


In [10]:
X.requires_grad_(True) # graident for backprop activated with requires_grad true
X.retain_grad() # leaf tensor is a root of the tree 

print(X.shape) 

torch.Size([500, 4])


In [11]:
# non- divident paying asset. 
# P + S - C = K * e^-r(T-t)
# Set strike price = 100

Strike = 100 # fixed hyper paramters. 

# Our case this is a fixed paramter. 

In [12]:
def euro_vanilla_call(x):
  d1 = (torch.log(x[0] / Strike) + (x[2] + 0.5 * x[3] ** 2) * x[1]) / (x[3] * torch.sqrt(x[1]))
  d2 = (torch.log(x[0] / Strike) + (x[2] - 0.5 * x[3] ** 2) * x[1]) / (x[3] * torch.sqrt(x[1])) # time x[1] with resepct to the time left not time. (- theta)
    
  call_t = (x[0] * torch.distributions.normal.Normal(0,1).cdf(d1) - Strike * torch.exp(-x[2] * x[1]) * torch.distributions.normal.Normal(0,1).cdf(d2)) 
  
  call = torch.nn.functional.relu(call_t) # to get rid of any negative prices
  return call
# https://pytorch.org/docs/stable/distributions.html - Normal( loc,scale)
#https://www.itl.nist.gov/div898/handbook/eda/section3/eda364.htm - loc and scale the same as mean and standard deviation.
# https://pytorch.org/docs/stable/generated/torch.log.html - returns natrual logarithem
# https://pytorch.org/docs/stable/generated/torch.sqrt.html - returns squart root


In [13]:
y = torch.zeros(size = (500 , 1)) # samples

for i in range(X.shape[0]):
  y[i] = euro_vanilla_call(X[i]) # runs through each row computation. 

print(y) # M samples dimension all in all. 

tensor([[2.27208824e+01],
        [2.64448738e+00],
        [6.00817680e+00],
        [1.13445139e+00],
        [1.38564720e+01],
        [4.16414928e+00],
        [1.25864067e+01],
        [6.48108292e+00],
        [1.70539360e+01],
        [4.95461197e+01],
        [5.22133350e-01],
        [1.24405594e+01],
        [2.48867950e+01],
        [5.03251038e+01],
        [1.16890182e+01],
        [9.22473145e+00],
        [3.56101418e+01],
        [0.00000000e+00],
        [1.41424825e-05],
        [1.52217827e+01],
        [3.11631203e+01],
        [9.11991882e+00],
        [0.00000000e+00],
        [5.06056213e+00],
        [6.10148430e-01],
        [2.52588081e+00],
        [3.01188324e-04],
        [2.30086517e+01],
        [2.53934784e+01],
        [3.36814523e-02],
        [1.74365559e+01],
        [3.53640914e-02],
        [4.33091819e-03],
        [5.51966286e+00],
        [8.12413025e+00],
        [3.16077194e+01],
        [1.93348999e+01],
        [2.71523514e+01],
        [7.7

In [14]:
f_o = torch.zeros(size = (500,4))

# First order DY/DX

for i in range(X.shape[0]):
  f_o[i] = torch.autograd.functional.jacobian( euro_vanilla_call, X[i])

print(f_o.shape) # m samples by n features 
  
print(f_o) # call option. Theta is always negative as depreciation with the value of the option. And measures the variation.

torch.Size([500, 4])
tensor([[ 0.68871599, 10.83909035, 38.51583862, 33.28545761],
        [ 0.20221856,  2.22321677, 16.54652023, 21.90766525],
        [ 0.39725709,  3.71162462, 39.11866760, 39.61265182],
        ...,
        [ 0.29155689,  3.50207663, 21.88513184, 27.09795952],
        [ 0.50912333,  5.90634251, 30.04350281, 32.91347885],
        [ 0.64543664,  5.17919683, 80.73455048, 51.79099274]])


In [15]:
s_o = torch.zeros(size = (500,4,4))

#Second Order Risk D2Y/ DX2

for i in range(X.shape[0]):
  s_o[i] = torch.autograd.functional.hessian(euro_vanilla_call , X[i])

print(s_o.shape) # m samples by n features by n features .
  
print(s_o) # take logarithems if really small. Monotonic.

torch.Size([500, 4, 4])
tensor([[[ 8.31418484e-03, -2.35627536e-02,  6.59594297e-01, -8.76964852e-02],
         [-2.35628486e-02, -7.17481613e+00,  4.47184486e+01,  2.45463295e+01],
         [ 6.59594417e-01,  4.47184601e+01,  2.49975967e+01, -3.05762863e+01],
         [-8.76967907e-02,  2.45463295e+01, -3.05763168e+01,  4.06529236e+00]],

        [[ 9.64909233e-03,  1.02882445e-01,  1.02531207e+00,  1.01059127e+00],
         [ 1.02882467e-01,  8.94269198e-02,  1.56337624e+01,  1.25302544e+01],
         [ 1.02531230e+00,  1.56337605e+01,  7.80325623e+01,  6.64510422e+01],
         [ 1.01059103e+00,  1.25302505e+01,  6.64510345e+01,  6.54969406e+01]],

        [[ 1.55720217e-02,  8.41536969e-02,  1.84763956e+00,  8.62057924e-01],
         [ 8.41535926e-02, -1.12068129e+00,  3.43516273e+01,  1.70959949e+01],
         [ 1.84763968e+00,  3.43516235e+01,  1.67030777e+02,  4.94311371e+01],
         [ 8.62058163e-01,  1.70960007e+01,  4.94312134e+01,  2.30631104e+01]],

        ...,

        

In [16]:
# Data Collection

x = X.type(torch.FloatTensor) #.to(device)
y = y.type(torch.FloatTensor) #.to(device)
f_o = f_o.type(torch.FloatTensor) #.to(device)
s_o = s_o.type(torch.FloatTensor) #.to(device)

print(X.shape)
print(y.shape)
print(f_o.shape)
print(s_o.shape)

torch.Size([500, 4])
torch.Size([500, 1])
torch.Size([500, 4])
torch.Size([500, 4, 4])


In [17]:
# Pre-processing for pytorch. 

full_data = torch.utils.data.TensorDataset(X , y, f_o, s_o)
train, val = random_split(full_data, [400, 100])
train_loader = DataLoader(train, batch_size = 32, shuffle=False , num_workers =0) # hardcore
val_loader = DataLoader(val, batch_size = 32 , shuffle=False , num_workers =0)

#X_train = train
#X_val, y_val, f_o_val, s_o_val = val

In [18]:
# Paramters and hyperparamters 
input_size = 4 # x features
hidden_size_1 = 40
hidden_size_2 = 20
output_size = 1 # y output of training 

#loss hyper- parameters 
alpha = 1
beta = 0.5
delta = 0.2

In [19]:
class SecondOrderNetwork(pl.LightningModule):
  
  def __init__(self):
    super(SecondOrderNetwork,self).__init__()
    self.layer_1 = nn.Linear(input_size, hidden_size_1) # linear layer
    self.layer_2 = nn.Linear(hidden_size_1, hidden_size_2)
    self.layer_3 = nn.Linear(hidden_size_2, output_size)

    self.softplus = nn.Softplus() # absolute 
    self.loss = nn.MSELoss() # Possibility try different loss. Difference in log -> MSE(log([y]) - log([predicted_y)]) -> solves small

    self.automatic_optimization = False # Turn off automatic optimization. Turns off backprop automatically. 
# Automatic optimization = False does manual optimization not autoamtic - https://pytorch-lightning.readthedocs.io/en/latest/common/optimizers.html
  def forward(self, x):
    # layer 1
    x = self.layer_1(x)
    x = self.softplus(x)

    # layer 2
    x = self.layer_2(x)
    x = self.softplus(x)

    # layer 3 
    x = self.layer_3(x)

    return x


  def training_step(self, train_batch, batch_idx):
    opt = self.optimizers() # access optimizers thats defined below in optimization function
    opt.zero_grad() # clear gradients from the preavious training set (so only keeps paramters that updated)

    x, y, f_o, s_o = train_batch
    predicted_y = self.forward(x)
    # loop over batch
    predicted_f_o = torch.zeros(size=(x.shape[0], 4)) #.to(device)
    for i in range(x.shape[0]): # x_shape is the batch size
      predicted_f_o[i] = torch.autograd.functional.jacobian(self.forward, x[i], create_graph=True, strict=True) # https://github.com/pytorch/pytorch/issues/46918 - Can show we need create_graph to be true as it backpropagets through the graph

    predicted_s_o = torch.zeros(size=(x.shape[0], 4, 4)) #.to(device) # when creating new variable put it on device
    for i in range(x.shape[0]): # x_shape is the batch size
      predicted_s_o[i] = torch.autograd.functional.hessian(self.forward, x[i], create_graph=True, strict=True)
    # create_graph = True makes derivatives differentiable through the chain using back prop. 
    # strict = True makes sure all inputs are connected to the outputs ortherwise an error will occur. If false it will be all zeros.
    loss = alpha * self.loss(y, predicted_y) + beta * self.loss( f_o, predicted_f_o) + delta * self.loss(s_o,  predicted_s_o) # change loss hyperparameters

    loss.backward(retain_graph=True) # Saves gradeints and not deleted them after first backprop . 

    opt.step() # Every batch update your paramters using optimizer step. 
    
    # logs 
    tensorboard_logs = {'train_loss' : loss}
    self.log('train_loss', loss)
    #print(loss)
    return {'loss': loss, 'log' : tensorboard_logs, 'train_loss': loss}

  def training_epoch_end(self, outputs):
    avg_loss = torch.stack([x['train_loss'] for x in outputs]).mean()
    tensorboard_logs = {'train_loss': avg_loss}
    print('train_loss: ', avg_loss , "epoch_number",self.current_epoch)
    {'avg_train_loss': avg_loss,'progress_bar': tensorboard_logs} # do not return for training epoch_end
    
  def validation_step(self, val_batch, batch_idx):
    x, y, f_o, s_o = val_batch
    predicted_y = self.forward(x)
    # loop over batch
    predicted_f_o = torch.zeros(size=(x.shape[0], 4)) #.to(device)
    for i in range(x.shape[0]): # validatio batch size
      predicted_f_o[i] = torch.autograd.functional.jacobian(self.forward, x[i], create_graph=True, strict=True)

    predicted_s_o = torch.zeros(size=(x.shape[0], 4, 4)) #.to(device)
    for i in range(x.shape[0]): # validation batch size
      predicted_s_o[i] = torch.autograd.functional.hessian(self.forward, x[i], create_graph=True, strict=True)

    loss = alpha * self.loss(y, predicted_y) + beta * self.loss( f_o, predicted_f_o) + delta * (self.loss(s_o, predicted_s_o))
    
    # logs
    self.log('val_loss', loss) # no optimization required for
  #  print( loss) 
    return {'val_loss' : loss}

  def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean() # calculates average loss for a epoch accounting for the batches
    tensorboard_logs = {'val_loss': avg_loss} # loggs on the graph for tensor boad
    print('val_loss: ', avg_loss, "epoch_number",self.current_epoch) # self.current_epoch logs the information for the epoch.
          
    return {'avg_val_loss': avg_loss,'progress_bar': tensorboard_logs}

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=1e-3)


In [20]:
trainer = pl.Trainer(max_epochs= 500)
# https://www.machinecurve.com/index.php/question/how-to-set-number-of-epochs-in-pytorch-lightning/
model = SecondOrderNetwork() #.to(device) # Trained. 

trainer.fit(model, train_loader, val_loader) 

# take lograithems 

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1293: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer"

  | Name     | Type     | Params
--------------------------------------
0 | layer_1  | Linear   | 200   
1 | layer_2  | Linear   | 820   
2 | layer_3  | Linear   | 21    
3 | softplus | Softplus | 0     
4 | loss     | MSELoss  | 0     
--------------------------------------
1.0 K     Trainable params
0         Non-trainable params
1.0 K     Total params
0.004     Total estimated model params size (MB)


val_loss:  tensor(2202.04199219) epoch_number 0


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:323: UserWarning: The number of training samples (13) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:398: LightningDeprecationWarning: One of the returned values {'log', 'train_loss'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  f"One of the returned values {set(extra.keys())} has a `grad_fn`. We will detach it automatically"


val_loss:  tensor(2983.45312500) epoch_number 0
train_loss:  tensor(2052.46191406) epoch_number 0


...

val_loss:  tensor(547.92803955) epoch_number 495
train_loss:  tensor(268.97607422) epoch_number 495


val_loss:  tensor(546.26611328) epoch_number 496
train_loss:  tensor(268.45547485) epoch_number 496


val_loss:  tensor(544.61743164) epoch_number 497
train_loss:  tensor(267.94012451) epoch_number 497


val_loss:  tensor(542.98205566) epoch_number 498
train_loss:  tensor(267.42950439) epoch_number 498


val_loss:  tensor(541.35931396) epoch_number 499
train_loss:  tensor(266.92340088) epoch_number 499



In [22]:
 %load_ext tensorboard 
 %tensorboard --logdir lightning_logs/
#%reload_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 192), started 2:30:02 ago. (Use '!kill 192' to kill it.)

<IPython.core.display.Javascript object>